# Solution

Solution, obviously, doesn't mean that this is the only correct solution. A representation of the problem could be more general (e.g. the datatype in this solution is fixed to `double`) or more strict (e.g. the value of Literal is just a string).

In [1]:
# Define the IR here

from eve import Node, Str
from pydantic import validator
from typing import List

class Expr(Node):
    pass

class Literal(Expr):
    value: Str

class BinaryOp(Expr):
    left: Expr
    right: Expr
    op: Str

# ------------- #
    
class Offset(Node):
    i: int
    j: int 

    @classmethod
    def zero(cls):
        return cls(i=0,j=0)

class FieldAccess(Expr):
    name: Str
    offset: Offset

class AssignStmt(Node):
    left: FieldAccess
    right: Expr

    @validator('left')
    def no_offset_in_assignment_lhs(cls, v):
        if v.offset.i != 0 or v.offset.j != 0:
            raise ValueError('Lhs of assignment must not have an offset')
        return v

class FieldDecl(Node):
    name: Str

# relative to domain
class Indent(Node):
    left: int
    right: int

class HorizontalLoop(Node):
    i_indent: Indent
    j_indent: Indent
    body: List[AssignStmt]

class Fun(Node):
    name: Str
    params: List[FieldDecl]
    horizontal_loops: List[HorizontalLoop]

In [2]:
# Programmatically construct a concrete IR for the Laplacian example

from devtools import debug

binop = BinaryOp(left=Literal(value="1"), right=Literal(value="1"), op="+")
debug(binop)

# ------------- #

lap_expr = BinaryOp(left=BinaryOp(left=Literal(value="-4"), right=FieldAccess(name="in", offset=Offset.zero()), op="*"), right=BinaryOp(left=BinaryOp(left=FieldAccess(name="in", offset=Offset(i=-1,j=0)),right=FieldAccess(name="in", offset=Offset(i=1,j=0)),op="+"),right=BinaryOp(left=FieldAccess(name="in", offset=Offset(i=0,j=-1)),right=FieldAccess(name="in", offset=Offset(i=0,j=1)),op="+"),op="+"), op="+")
assign = AssignStmt(left=FieldAccess(name="out", offset=Offset.zero()), right=lap_expr)
hloop = HorizontalLoop(i_indent=Indent(left=1, right=1),j_indent=Indent(left=1, right=1), body=[assign])

lap=Fun(name="lap", params=[FieldDecl(name="out"), FieldDecl(name="in")], horizontal_loops=[hloop])

debug(lap)

<ipython-input-1-b15736de57c7>:6 <module>
    binop: BinaryOp(
        id_='BinaryOp_3',
        left=Literal(
            id_='Literal_1',
            value='1',
        ),
        right=Literal(
            id_='Literal_2',
            value='1',
        ),
        op='+',
    ) (BinaryOp)
<ipython-input-1-b15736de57c7>:16 <module>
    lap: Fun(
        id_='Fun_28',
        name='lap',
        params=[
            FieldDecl(
                id_='FieldDecl_26',
                name='out',
            ),
            FieldDecl(
                id_='FieldDecl_27',
                name='in',
            ),
        ],
        horizontal_loops=[
            HorizontalLoop(
                id_='HorizontalLoop_25',
                i_indent=Indent(
                    id_='Indent_23',
                    left=1,
                    right=1,
                ),
                j_indent=Indent(
                    id_='Indent_24',
                    left=1,
                    right=1,
        

In [3]:
# Code generator

from eve.codegen import FormatTemplate, TemplatedGenerator
# ------------- #
from eve.codegen import MakoTemplate

class LIR_to_cpp(TemplatedGenerator):
    Literal = FormatTemplate("{value}")
    BinaryOp = FormatTemplate("({left}{op}{right})")

# ------------- #

    Offset = FormatTemplate("[i+{i}][j+{j}]")

    FieldAccess = FormatTemplate("{name}{offset}")

    AssignStmt = FormatTemplate("{left} = {right};")

    FieldDecl = FormatTemplate("Field& {name}")

    #TODO use visit for Indent?

    # Using domain and i, j by hard-coded string is not a good design!
    HorizontalLoop = MakoTemplate("""for(std::size_t i = ${_this_node.i_indent.left}; i < domain[0] - ${_this_node.i_indent.right}; ++i) {
            for(std::size_t j = ${_this_node.j_indent.left}; j < domain[1] - ${_this_node.j_indent.right}; ++j) {
                ${''.join(body)}
            }
        }""")

    Fun = MakoTemplate("""void ${name}(Domain domain, ${','.join(params)}){
        ${''.join(horizontal_loops)}
        }""")

In [4]:
# Call the generator

import os
from eve.codegen import format_source

cppcode = LIR_to_cpp.apply(lap)
formatted_code = format_source("cpp", cppcode, style="LLVM")
print(formatted_code)

output_file = "generated.hpp"
with open(output_file, "w+") as output:
    output.write(formatted_code)

void lap(Domain domain, Field &out, Field &in) {
  for (std::size_t i = 1; i < domain[0] - 1; ++i) {
    for (std::size_t j = 1; j < domain[1] - 1; ++j) {
      out[i + 0][j + 0] =
          ((-4 * in[i + 0][j + 0]) + ((in[i + -1][j + 0] + in[i + 1][j + 0]) +
                                      (in[i + 0][j + -1] + in[i + 0][j + 1])));
    }
  }
}


In [5]:
!g++ driver.cc -o lap
!./lap

  0   0   0   0   0 
  0  -2   0  -2   0 
  0   0  -4   0   0 
  0  -2   0  -2   0 
  0   0   0   0   0 
